In [8]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
# reading url
httpObject=urlopen('https://www.flipkart.com/q/best-laptops-under-rs-50000')
webdata = httpObject.read()
print(webdata)

HTTPError: HTTP Error 500: Internal Server Error

In [3]:
soup1 = soup(webdata)

NameError: name 'webdata' is not defined

In [ ]:
pages_link = soup1.findAll('a',{'class':'ge-49M'})
domain = 'https://www.flipkart.com/q/best-laptops-under-rs-50000?page='
for i in range(2,36):
    link= domain + str(i)
    page_data = urlopen(link)
    webdata1 = page_data.read()
    webdata += webdata1

In [ ]:
soupdata = soup(webdata, 'html.parser')
type(soupdata)

In [ ]:
soupdata

In [ ]:
# all containers data
containers=soupdata.findAll('div',{'class':'_2kHMtA'})
print(type(containers),len(containers))
# print(containers)

In [ ]:
# data of 1st container
containers[0]

In [ ]:
# data of any in range containers
containers[50]

In [ ]:
f =open('laptops_info.csv','wb')
f.write('ProductName,Stars,Ratings,Reviews,CurrentPrice,MRP,Processor,RAM,Storage,ImageURL\n'.encode())
import re
for container in containers:
#     finding product name
    product = container.findAll('div',{'class':'_4rR01T'})
    ProductName = product[0].text.split('-')[0].strip()
    
    
#     finding stars
    star = container.find('div',{'class':'_3LWZlK'})
    try:
        Stars = star.text
    except:
        Stars = 0
        
#     finding ratings n reviews
    Rating= container.find('span',{'class':'_2_R_DZ'})
    
    try:
        ratRev = re.findall('\d+,?\d*',Rating.text)
        Ratings=ratRev[0].replace(',','')
        Reviews=ratRev[1].replace(',','')
        
        
    except:
        Ratings=0
        Reviews=0
#     print(Ratings,Reviews)
    
#     finding current price N mrp price
    CurrentPrice = container.find('div',{'class':'_30jeq3 _1_WHN1'}).text.replace(',','').replace('₹','')
   
    
    mrp=container.find('div',{'class':'_3I9_wc _27UcVY'})
    try:
        
        MRP=mrp.text.replace(',','').replace('₹','')
    except:
        MRP=0

#         finding information about product
    info = container.findAll('li',{'class':'rgWa7D'})
    Processor=info[0].text
    RAM = info[1].text
    Storage = info[3].text
    
    
    Image = container.img
    ImageURL = Image.get('src')
    print(ProductName, Stars, Ratings, Reviews, CurrentPrice, MRP, Processor, RAM, Storage, ImageURL)
    f.write(f"{ProductName},{Stars},{Ratings},{Reviews},{CurrentPrice},{MRP},{Processor},{RAM},{Storage},{ImageURL}\n".encode())
    print('\n')
f.close()

In [ ]:
df = pd.read_csv('laptops_info.csv')
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.drop(['Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16'], axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='all')

In [ ]:
df.corr()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# finding product having highest ratings
df[df['Ratings']==max(df['Ratings'])]

In [ ]:
# top 5 rated laptops
top5ratedlaptops = df.sort_values(by='Ratings',ascending=False).head()

In [ ]:
top5ratedlaptops

In [ ]:
# laptops from lowest price
df.sort_values(by='CurrentPrice',ascending=True).head()

In [ ]:
# if we want only few columns from dataframe
smalldf = df[['ProductName','CurrentPrice','MRP']]
smalldf

In [ ]:
# if we want price in range
midrange = df[(df['CurrentPrice']>20000) & (df['CurrentPrice']<45000)]

In [ ]:
midrange

In [ ]:
# for user input
ram = input()
df[df['RAM']==ram]

In [ ]:
# user input
ram = input("RAM? : ")
storage = input("Storage? : ")
processor = input("Processor? : ")
# df[(df.RAM == ram) & (df.Storage == storage) & (df.Processor == processor)]

In [ ]:
df.RAM == ram
df.Storage == storage
df.Processor == processor

In [ ]:
# for getting above names

df[df.RAM==ram]['ProductName']

In [ ]:
# for all above condition in one
df[(df.RAM == ram) & (df.Storage == storage) & (df.Processor == processor)]

In [ ]:
# plotting bar graph
# without remvoing duplicates
plt.figure(figsize=(15,7))
plt.bar(x=top5ratedlaptops['ProductName'], height=top5ratedlaptops['Ratings'])
plt.title("Highest Rating Laptops", fontsize=30)
plt.xlabel("Laptops Name",fontsize=15)
plt.ylabel("Total Ratings",fontsize=15)
plt.xticks(rotation=270)
plt.show()

In [ ]:
#  affordable laptops bar graph
top10affordable = df.sort_values(by='CurrentPrice',ascending=False).head(10)

In [ ]:
plt.figure(figsize=(12,7))
plt.bar(x=top10affordable['ProductName'], height=top10affordable['CurrentPrice'])
plt.title("Lowest Priced Laptops", fontsize=30)
plt.xlabel("Laptops Name",fontsize=15)
plt.ylabel("Current Price",fontsize=15)
plt.xticks(rotation=270)
plt.show()


In [ ]:
# scatter matrix
pd.plotting.scatter_matrix(df,figsize=[15,15],alpha=0.5,diagonal='kde')
plt.show()

In [ ]:

# cross tabulation
pd.crosstab(df['CurrentPrice'],df['MRP'],rownames=['CurrentPrice'],colnames=['MRP'])

In [ ]:
# detecting missing values
df.isna()

In [ ]:
# finding outliers
def find_outliers(ds,col):
    Q1=ds[col].quantile(0.25)
    Q3=ds[col].quantile(0.75)
    IQR=Q3-Q1
    low_val=Q1-1.5*(IQR)
    high_val=Q3+1.5*(IQR)
    ds=ds.loc[(ds[col] < low_val) | (ds[col] > high_val)]
    return ds

In [ ]:
find_outliers(df,'CurrentPrice')

In [ ]:
#  remove outliers
def remove_outliers(ds,col):
    Q1=ds[col].quantile(0.25)
    Q3=ds[col].quantile(0.75)
    IQR=Q3-Q1
    low_val=Q1-1.5*(IQR)
    high_val=Q3+1.5*(IQR)
    ds_out=ds.loc[(ds[col] >= low_val) & (ds[col] <= high_val)]
    return ds_out

In [ ]:
remove_outliers(df,'CurrentPrice')

In [ ]:
# removing duplicates
df.drop_duplicates()

In [ ]:
df.reset_index(drop=True)

In [ ]:
# boxplot
dataset=pd.read_csv('laptops_info.csv')
plt.boxplot(dataset['CurrentPrice'],widths=0.75)
plt.show()

In [ ]:
# scatter plot
x=df['Stars']
y=df['Reviews']
plt.scatter(x,y,s=[40],marker='^',c='green')
plt.xlabel('reviews')
plt.ylabel('ratings')

In [ ]:
# regplot
dataset1=pd.read_csv('laptops_info.csv')
dataset.corr()
ax=sns.regplot(x='CurrentPrice',y='MRP',data=dataset)
ax.set(xlabel='currentprice',ylabel='mrp')
ax.set_title('currentprice vs mrp')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
import pandas as pd
df=pd.read_csv('laptops1_info.csv')

In [ ]:
df

In [ ]:
df.drop(['ModelName'],axis=1)

In [ ]:
df['ProcessorName']=df['ProcessorName'].str.strip()
df['ProcessorName']=df['ProcessorName'].replace(' i7','I7')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df=pd.get_dummies(df,drop_first=True)

In [ ]:
df

In [ ]:
# training and testing
from sklearn.model_selection import train_test_split
Y=df['Price']
X=df.drop('Price',axis=1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
print('x_train',X_train.shape)
print('x_test',X_test.shape)
print('y_train',Y_train.shape)
print('y_test',Y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
model=lr.fit(X_train,Y_train)

In [ ]:
X_test

In [ ]:
Y_train

In [ ]:
Y_pred=lr.predict(X_test)
# len(Y_pred)
Y_pred

In [ ]:
[[8, 1024, 256, 0, 4, 0, 1, 0, 0, 0,0, 0, 0, 0,0,0,0, 0,0,0, 0,0, 0, 0,0, 0,1,0, 0, 0,1, 0, 0,0]]

In [ ]:
Y_pred=lr.predict([[8, 1024, 256, 0, 4, 0, 1, 0, 0, 0,0, 0, 0, 0,0,0,0, 0,0,0, 0,0, 0, 0,0, 0,1,0, 0, 0,1, 0, 0,0]])
Y_pred

In [ ]:
lr.coef_

In [ ]:
len(lr.coef_)

In [ ]:
lr.intercept_

In [ ]:
from sklearn import metrices

In [ ]:
print("MAE",metrices.mean_absolute_error(Y_test,Y_pred))
print("MSE",metrices.mean_squared_error(Y_test,Y_pred))